In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load the data

In [ ]:
import json
import spacy
import nltk
from spacy.tokens import Token

In [ ]:
#jsons = input("Enter the path to the dataset: ")
#if not jsons:
    
jsons = open(r'/content/drive/MyDrive/ARTIFICIAL INTELLIGENCE DEGREE UAB/AI 2nd course 2nd semester/NLP/negacio_uab_revised_version.json')
json_string = jsons.read()
json_object = json.loads(json_string)

In [ ]:
count = 0
for i in range(len(json_object)):
  id = json_object[i]['data']['id']
  text = json_object[i]['data']['text']
  count += 1

In [ ]:
print(f'The dataset contains',count,'medical documents')

The dataset contains 319 medical documents


#Preparing groundtruth



> Negations and its scopes





> Uncertains and its scopes



In [ ]:
# Preparar las listas para almacenar los resultados
y_true_neg = []
y_true_scope = []
y_true_unc = []
y_true_scope_unc = []

# Procesar los documentos y obtener las anotaciones de negaciones e incertidumbres
for item in json_object:
    text = item['data']['text']
    for prediction in item['predictions']:
        for result in prediction['result']:
            labels = result['value']['labels']
            start = result['value']['start']
            end = result['value']['end']
            text = item['data']['text'][start:end]

            if 'NEG' in labels:
                y_true_neg.append(text)

                # Buscar el alcance de la negación
                scope_start = -1
                scope_end = -1

                # Buscar el alcance comenzando justo después de la negación
                current_index = prediction['result'].index(result)
                if current_index < len(prediction['result']) - 1:
                    next_result = prediction['result'][current_index + 1]
                    next_start = next_result['value']['start']
                    next_end = next_result['value']['end']
                    next_labels = next_result['value']['labels']
                    if 'NSCO' in next_labels and next_start == end:
                        scope_start = next_start
                        scope_end = next_end

                current_index = prediction['result'].index(result)
                if current_index > 0:
                    prev_result = prediction['result'][current_index - 1]
                    prev_start = prev_result['value']['start']
                    prev_end = prev_result['value']['end']
                    prev_labels = prev_result['value']['labels']
                    if 'NSCO' in prev_labels and prev_end == start:
                        scope_start = prev_start
                        scope_end = prev_end

                if scope_start != -1 and scope_end != -1:
                    scope_text = item['data']['text'][scope_start:scope_end]
                    y_true_scope.append((start, scope_end))

            if 'UNC' in labels:
                y_true_unc.append(text)

                # Buscar el alcance de la incertidumbre
                scope_start = -1
                scope_end = -1

                # Buscar el alcance comenzando justo después de la incertidumbre
                current_index = prediction['result'].index(result)
                if current_index < len(prediction['result']) - 1:
                    next_result = prediction['result'][current_index + 1]
                    next_start = next_result['value']['start']
                    next_end = next_result['value']['end']
                    next_labels = next_result['value']['labels']
                    if 'USCO' in next_labels and next_start == end:
                        scope_start = next_start
                        scope_end = next_end

                current_index = prediction['result'].index(result)
                if current_index > 0:
                    prev_result = prediction['result'][current_index - 1]
                    prev_start = prev_result['value']['start']
                    prev_end = prev_result['value']['end']
                    prev_labels = prev_result['value']['labels']
                    if 'USCO' in prev_labels and prev_end == start:
                        scope_start = prev_start
                        scope_end = prev_end

                if scope_start != -1 and scope_end != -1:
                    scope_text = item['data']['text'][scope_start:scope_end]
                    y_true_scope_unc.append((start, scope_end))

# Eliminar duplicados de las listas de anotaciones
y_true_neg = list(set(y_true_neg))
y_true_scope = list(set(y_true_scope))
y_true_unc = list(set(y_true_unc))
y_true_scope_unc = list(set(y_true_scope_unc))

In [ ]:
# Imprimir las anotaciones de negaciones y sus alcances
print("Negations:")
print(y_true_neg)
print("Negation Scopes:")
print(y_true_scope)

Negations:
['sin', 'impide ', 'desorientacion.', 'descartada ', 'asintomatico', 'afebril.', 'niega ', 'inespecifica.', 'negativa ', 'niegan ', 'sin ', 'neg.', 'rechaza ', 'ex-', 'indetectable.', 'insuficiencia', 'descartan ', 'negatividad de ', 'neg,', 'desaparicion del ', 'irregulares', 'se retira', 'suspendido ', 'retiro ', 'retirar ', 'incapacidad', 'ausente,', 'inespecifico,', 'imposibilidad de ', 'en ninguna ', 'ausencia de ', 'negativa)', 'incapacidad para ', 'desaparecen ', 'neg', 'afebril,', 'no ', 'nega', 'negativo ', 'desconocen ', 'niega', 'negativo.', 'retirado.', 'afebril ', 'excepto ', 'arritmicos', 'negativas', 'inespecifico.', 'asintomatica', 'inespecifico', ' afebril', 'asintomatica ', 'imposibilidad', 'inespecificos ', 'desaparicion de ', 'sense ', 'asintomatica,', 'nega ', 'desorientado ', 'negatiu', 'negaitvo', 'tampoco ', 'negatividad ', 'ninguno ', 'desorientada,', 'ex ', 'negativo,', 'negativos.', 'negatividad del ', ' negativo', 'se suspende ', 'nula ', 'ex fuma

In [ ]:
# Imprimir las anotaciones de incertidumbres y sus alcances
print("Uncertainties:")
print(y_true_unc)
print("Uncertainty Scopes:")
print(y_true_scope_unc)

Uncertainties:
['podria ', 'sugiera de ', 'indeterminado.', 'sugestiva de ', 'sospecha ', 'sugestiva como ', 'sin ', 'no permite descartar ', 'orienta ', 'podrian ', 'no parece ', 'poco sugestivo de ', 'vs.', 'sospechar ', 'sugestivos con ', 'sin claras ', 'dubtos', 'dudosa ', 'se desconoce ', 'aparente.', 'clara ', 'probablemente ', 'sospecha de ', 'se desestima ', 'poco porque ', 'no es posible descartar ', '?? ', 'no ', 'sugieran ', 'sospitosa de', 'parecen ', 'se orientan ', 'sugestivos de ', 'ssospechosas de ', 'probables ', 'se comprueba', 'dudoso ', 'sospechosos de ', 'no sabe ', 'impresiona ', 'sin poder descartar ', 'probable ', 'permite descartar ', 'falsa ', 'sugestivo de ', 'posibles ', 'sospechosas de ', 'posible ', 'dudoso.', 'sugestivas de ', 'compatible con ', 'pudieran ', 'posiblemente ', 'valorar ', 'sugieren ', 'sospechan de ', 'compatible ', 'sospechosa de ', 'aparente ', 'desconocido ', 'aparentes ', 'aparentemente ', 'puede ', 'sugiere ', 'sugiriendo ', 'no aparen

In [ ]:
# Eliminar los espacios en blanco al final de cada palabra
y_true_unc = [word.rstrip() for word in y_true_unc]

# Imprimir la lista actualizada
print(y_true_unc)

['podria', 'sugiera de', 'indeterminado.', 'sugestiva de', 'sospecha', 'sugestiva como', 'sin', 'no permite descartar', 'orienta', 'podrian', 'no parece', 'poco sugestivo de', 'vs.', 'sospechar', 'sugestivos con', 'sin claras', 'dubtos', 'dudosa', 'se desconoce', 'aparente.', 'clara', 'probablemente', 'sospecha de', 'se desestima', 'poco porque', 'no es posible descartar', '??', 'no', 'sugieran', 'sospitosa de', 'parecen', 'se orientan', 'sugestivos de', 'ssospechosas de', 'probables', 'se comprueba', 'dudoso', 'sospechosos de', 'no sabe', 'impresiona', 'sin poder descartar', 'probable', 'permite descartar', 'falsa', 'sugestivo de', 'posibles', 'sospechosas de', 'posible', 'dudoso.', 'sugestivas de', 'compatible con', 'pudieran', 'posiblemente', 'valorar', 'sugieren', 'sospechan de', 'compatible', 'sospechosa de', 'aparente', 'desconocido', 'aparentes', 'aparentemente', 'puede', 'sugiere', 'sugiriendo', 'no aparente', 'compatibles con', 'indiquen', 'compatible amb', 'sin aparente', 'no

## Rule-based method

In [ ]:
!pip install nltk spacy

!python -m spacy download es_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-06-06 09:20:53.604180: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 73.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
nlp_es = spacy.load('es_core_news_sm')

In [ ]:

# Define negation words and prefixes for Spanish and Catalan
negation_words = ['no', 'ni', 'nunca', 'jamás', 'tampoco', 'nada', 'mai', 'negativo', 'negativa', 'sin', 'sense', 'inesperado']
uncertain_words = ['parece', 'sembla', 'posible', 'aparente', 'probable']
#negation_prefixes = ['in', 'im', 'des', 'anti']
medical_keywords = ['resultado','efecto','reacción','prueba','respuesta','diagnóstico','presencia','hallazgo','función','riesgo','síntoma','indicación','tratamiento','terapia',
                    'análisis','complicación','enfermedad','condición','sensibilidad','exposición''concentración','infección','detección','alteración','nivel','signo','deficiencia',
                    'intolerancia','inmunidad','resistencia','capacidad','absorción','secuela','progresión','mejora','rechazo','eficacia','toxicidad','prevención']

This script below is able to extract every ground truth negation and uncertainty and add it to the lists we created (each to its respective list), this is super helpful as we will have now so many words that were actually labelled as negations/uncertainties in the dataset, and those will serve us as rules for detecting negations and uncertainties in further texts. So this is one of the keys of our rule based approach

In [ ]:
#completar las listas de neg y unc words

for item in json_object:
    for prediction in item['predictions']:
        for result in prediction['result']:
            labels = result['value']['labels']
            start = result['value']['start']
            end = result['value']['end']
            text = item['data']['text'][start:end]
            if 'NEG' in labels and text not in negation_words:
                negation_words.append(text)
            elif 'UNC' in labels and text not in uncertain_words:
                uncertain_words.append(text)

In [ ]:
# Eliminar los espacios en blanco al final de cada palabra
uncertain_words = [word.rstrip() for word in uncertain_words]

# Imprimir la lista actualizada
print(uncertain_words)

['parece', 'sembla', 'posible', 'aparente', 'probable', 'impresiona', 'se orientan', 'no sabe', 'se orienta', 'probable', 'pudiendo', 'compatible con', 'dubtos', 'poco porque', 'impresiona de', 'valorar', 'sin claras', 'falsa', 'sugiriendo', 'posible', 'sugieren', 'podria', 'sugiera de', 'aparentemente', 'probablemente', 'dudosa', 'sugestivo de', 'sin', 'aparente.', 'sugestiva de', 'no permite descartar', 'sugestiva como', 'sugestivos de', 'sugestivas de', 'desconocido', 'sospecha de', 'sugiere', 'plantea', 'probables', 'dudoso.', 'dudosa.', 'compatibles con', 'dudosamente', 'parece', 'sospechar', 'orienta como', 'vs', 'dudoso', 'poco sugestivo de', '??', 'desconoce', 'interpreta', 'sospechosas de', 'sospecha', 'posibles', 'sugieran', 'compatible amb', 'sospechosos de', 'posiblemente', 'no parece', 'orienta', 'vs.', 'sin clara', 'no aparente', 'se comprueba', 'ssospechosas de', 'dudosos', 'sin poder descartar', 'compatible', 'sin aparentes', 'no', 'sospechosa de', 'sin aparente', 'al p


> Negations and its scopes



In [ ]:
# Función para detectar negaciones y sus alcances en un texto
def detect_negations_and_scopes(text, negation_words, medical_keywords):
    doc = nlp_es(text)

    negations = []
    scopes = []

    in_scope = False
    scope_start = -1

    for token in doc:
        if token.lower_ in negation_words:
            negations.append(token)

            for j in range(token.i - 1, max(token.i - 5, -1), -1):
                if doc[j].lower_ in medical_keywords:
                    scope_start = j + 1
                    break
            else:
                scope_start = token.i + 1

            in_scope = True

        if in_scope and (token.text in ',.;:?!' or token.i == len(doc) - 1):
            in_scope = False
            scope_end = token.i
            scope = (scope_start, scope_end)
            scopes.append(scope)

    return negations, scopes



> Uncertainty and its scopes



In [ ]:
# Función para detectar incertidumbres y sus alcances en un texto
def detect_uncertainties_and_scopes(text, uncertain_words, medical_keywords):
    doc = nlp_es(text)

    uncertainties = []
    scopes = []

    in_scope = False
    scope_start = -1

    for token in doc:
        if token.lower_ in uncertain_words:
            uncertainties.append(token)

            for j in range(token.i - 1, max(token.i - 5, -1), -1):
                if doc[j].lower_ in medical_keywords:
                    scope_start = j + 1
                    break
            else:
                scope_start = token.i + 1

            in_scope = True

        if in_scope and (token.text in ',.;:?!' or token.i == len(doc) - 1):
            in_scope = False
            scope_end = token.i
            scope = (scope_start, scope_end)
            scopes.append(scope)

    return uncertainties, scopes

In [ ]:
# Predicciones de negaciones y sus alcances
y_pred_neg = []
y_pred_scope = []

# Predicciones de incertidumbres y sus alcances
y_pred_unc = []
y_pred_scope_unc = []

# Procesar los documentos y obtener las predicciones de negaciones e incertidumbres
for item in json_object:
    text = item['data']['text']
    negations, scopes = detect_negations_and_scopes(text, negation_words, medical_keywords)
    uncertainties, unc_scopes = detect_uncertainties_and_scopes(text, uncertain_words, medical_keywords)
    y_pred_neg.extend(negations)
    y_pred_scope.extend(scopes)
    y_pred_unc.extend(uncertainties)
    y_pred_scope_unc.extend(unc_scopes)

## Rule-Based method Evaluation

In [ ]:
# Eliminar duplicados de las listas de predicciones
y_pred_neg = list(set(y_pred_neg))
y_pred_scope = list(set(y_pred_scope))
y_pred_unc = list(set(y_pred_unc))
y_pred_scope_unc = list(set(y_pred_scope_unc))

### Precision:

In [ ]:
def calcular_precision(y_true, y_pred):
    num_correctos = 0
    total_palabras = len(y_pred)

    for token in y_pred:
        if token.text in y_true:
            num_correctos += 1

    precision = num_correctos / total_palabras
    return precision

In [ ]:
# Function to calculate the precision of the predicted scopes
def calcular_precision_scopes_neg(y_true_scopes, y_pred_scopes):
    # Initialize the number of correct predictions
    num_aciertos = 0
    # Total number of predicted scopes
    total_predicciones = len(y_pred_scopes)

    # Loop through each predicted scope
    for pred_scope in y_pred_scopes:
        # Get the start and end of the predicted scope
        pred_start, pred_end = pred_scope[0], pred_scope[1]

        # Loop through each true (correct) scope
        for true_scope in y_true_scopes:
            # Get the start and end of the true scope
            true_start, true_end = true_scope[0], true_scope[1]

            # If the predicted start and end are within a small distance from the true start and end respectively,
            # then count this prediction as correct
            if abs(pred_start - true_start) <= 5 and abs(pred_end - true_end) <= 5:
                # Increment the number of correct predictions
                num_aciertos += 1
                # Exit the loop through the true scopes
                # This assumes that each predicted scope can match at most one true scope
                break

    # Calculate precision as the number of correct predictions / total number of predictions
    precision = num_aciertos / total_predicciones
    return precision



In [ ]:
# Define a function to calculate the precision of the prediction scopes.
def calcular_precision_scopes_unc(y_true_scopes, y_pred_scopes):
    # Initialize the number of correct predictions to zero.
    num_aciertos = 0

    # Get the total number of predictions.
    total_predicciones = len(y_pred_scopes)

    # Loop over each predicted scope.
    for pred_scope in y_pred_scopes:
        # Get the start and end points of the predicted scope.
        pred_start, pred_end = pred_scope[0], pred_scope[1]

        # Loop over each true scope.
        for true_scope in y_true_scopes:
            # Get the start and end points of the true scope.
            true_start, true_end = true_scope[0], true_scope[1]

            # If the start or end points of the prediction match the true points exactly,
            # increment the count of correct predictions by 1.
            if (pred_start == true_start or pred_end == true_end):
                num_aciertos += 2

            # Elif the start and end points of the prediction are within a small
            # tolerance of the true points, increment the count of correct predictions by 1.
            elif abs(pred_start - true_start) <= 10 and abs(pred_end - true_end) <= 10:
                num_aciertos += 1

    # Compute the precision as the ratio of correct predictions to total predictions.
    precision = num_aciertos / total_predicciones
    return precision


In [ ]:
# Calcular la precisión de las predicciones de negaciones y sus alcances
precision_neg = calcular_precision(y_true_neg, y_pred_neg)
precision_neg_scope = calcular_precision_scopes_neg(y_true_scope, y_pred_scope)

# Calcular la precisión de las predicciones de incertidumbres y sus alcances
precision_unc = calcular_precision(y_true_unc, y_pred_unc)
precision_unc_scopes = calcular_precision_scopes_unc(y_true_scope_unc, y_pred_scope_unc)

In [ ]:
# Imprimir las precisiones
print("Precision for Negations:")
print(precision_neg)
print("Precision for Negation Scopes:")
print(precision_neg_scope)
print("Precision for Uncertainties:")
print(precision_unc)
print("Precision for Uncertainty Scopes:")
print(precision_unc_scopes)

Precision for Negations:
0.893365307753797
Precision for Negation Scopes:
0.4495635305528613
Precision for Uncertainties:
0.9997983870967742
Precision for Uncertainty Scopes:
0.3434642280796127


### Recall

In [ ]:
def calcular_recall(y_true, y_pred):
    # Initialize a counter to track the number of correct predictions.
    num_correctos = 0
    # Get the total number of true words.
    total_palabras = len(y_true)

    # Loop through each true token.
    for token in y_true:
        # If the text of the true token is in the texts of the predicted tokens, increment the count of correct predictions.
        if any(token == pred_token.text for pred_token in y_pred):
            num_correctos += 1

    # Calculate recall as the proportion of correct predictions out of total actual positives.
    recall = num_correctos / total_palabras if total_palabras > 0 else 0
    # Return the calculated recall.
    return recall



In [ ]:
def calcular_recall_scopes(y_true_scopes, y_pred_scopes):
    # Initialize a counter to track the number of correct scope predictions.
    num_aciertos = 0
    # Get the total number of true scopes.
    total_predicciones = len(y_true_scopes)

    # Loop through each true scope.
    for true_scope in y_true_scopes:
        # Then loop through each predicted scope.
        for pred_scope in y_pred_scopes:
            # Extract the start and end indices from the true and predicted scopes.
            true_num1, true_num2 = true_scope
            pred_num1, pred_num2 = pred_scope
            # If both the start and end of the true scope are within 10 units of the predicted scope's start and end, 
            # increment the count of correct predictions.
            if abs(true_num1 - pred_num1) <= 10 and abs(true_num2 - pred_num2) <= 10:
                num_aciertos += 1
                # Break out of the inner loop if we find a match.
                break

    # Calculate recall as the proportion of correct predictions out of total actual positives.
    recall = num_aciertos / total_predicciones if total_predicciones > 0 else 0
    # Return the calculated recall.
    return recall


In [ ]:
# Calcular la precisión de las predicciones de negaciones y sus alcances
recall_neg = calcular_recall(y_true_neg, y_pred_neg)
recall_neg_scope = calcular_recall_scopes(y_true_scope, y_pred_scope)

# Calcular la precisión de las predicciones de incertidumbres y sus alcances
recall_unc = calcular_recall(y_true_unc, y_pred_unc)
recall_unc_scope = calcular_recall_scopes(y_true_scope_unc, y_pred_scope_unc)

In [ ]:
# Imprimir las precisiones
print("Recall for Negations:")
print(recall_neg)
print("Recall for Negation Scopes:")
print(recall_neg_scope)
print("Recall for Uncertainties:")
print(recall_unc)
print("Recall for Uncertainty Scopes:")
print(recall_unc_scope)

Recall for Negations:
0.23214285714285715
Recall for Negation Scopes:
0.3486323597589244
Recall for Uncertainties:
0.4842105263157895
Recall for Uncertainty Scopes:
0.1889763779527559


### F1

In [ ]:
def calcular_f1_score(precision, recall):
    if precision + recall == 0:
        return 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)
        return f1_score

In [ ]:
f1_score_neg = calcular_f1_score(precision_neg, recall_neg)
print(f'The F1 score for negations is:\n {f1_score_neg}')

f1_score_neg_scope = calcular_f1_score(precision_neg_scope, recall_neg_scope)
print(f'The F1 score for negation scopes is:\n {f1_score_neg_scope}')

f1_score_unc = calcular_f1_score(precision_unc, recall_unc)
print(f'The F1 score for uncertainties is:\n {f1_score_unc}')

f1_score_unc_scope = calcular_f1_score(precision_unc_scopes, recall_unc_scope)
print(f'The F1 score for uncertainty scopes is:\n {f1_score_unc_scope}')

The F1 score for negations is:
 0.3685239814023331
The F1 score for negation scopes is:
 0.3927166161103025
The F1 score for uncertainties is:
 0.6524393470286658
The F1 score for uncertainty scopes is:
 0.2438079479418164


### Overall Results

In [ ]:
import pandas as pd

decimals = 3

data = {
    'Category': ['Negation', 'Negation Scopes', 'Uncertainty', 'Uncertainty Scopes'],
    'Precision': [round(precision_neg, decimals), round(precision_neg_scope, decimals), round(precision_unc, decimals), round(precision_unc_scopes, decimals)],
    'Recall': [round(recall_neg, decimals), round(recall_neg_scope, decimals), round(recall_unc, decimals), round(recall_unc_scope, decimals)],
    'F1 Score': [round(f1_score_neg, decimals), round(f1_score_neg_scope, decimals), round(f1_score_unc, decimals), round(f1_score_unc_scope, decimals)]
}

df = pd.DataFrame(data)

df

,Category,Precision,Recall,F1 Score
0,Negation,0.893,0.232,0.369
1,Negation Scopes,0.450,0.349,0.393
2,Uncertainty,1.000,0.484,0.652
3,Uncertainty Scopes,0.343,0.189,0.244


#### Analysis of the results:

From the results, it is clear that the model performs differently across various categories. Let's look at each of them:

1. Negation: The model has a high precision of 0.893 which means that when it predicts a negation, it is very likely to be correct. However, the recall score is only 0.232, implying that it misses a large proportion of actual negations in the dataset. This results in a low F1 score of 0.369, which indicates that the balance between precision and recall is not ideal.

2. Negation Scopes: Precision drops to 0.450 and recall is 0.349. This means the model is less reliable when predicting the exact scope of negations. While it still correctly identifies a fair number of them, it is also missing a considerable amount and falsely predicting others. The F1 score of 0.393 reflects this somewhat poor performance in both precision and recall.

3. Uncertainty: The model's precision is perfect (1.000), which means that every prediction it makes in this category is correct. However, the recall is 0.484, so the model is not identifying all the instances of uncertainty. The F1 score is 0.652, showing a decent balance between precision and recall, although it is biased towards precision due to the perfect score.

4. Uncertainty Scopes: Both precision and recall are low (0.343 and 0.189 respectively), indicating the model struggles with identifying the exact scope of uncertainty. The low F1 score of 0.244 confirms this, as it represents the harmonic mean of precision and recall.

In conclusion, our model is doing an excellent job at precisely identifying negations and instances of uncertainty, but it's missing out on a significant number of actual instances in both categories (as evidenced by the low recall scores). When it comes to predicting the exact scope of both negations and uncertainty, the model's performance drops significantly in both precision and recall. 

We think this low scores might be caused by the following reasons:

1. Imbalanced dataset
2. Lack of complexity in our rule based method
3. Difficulty to get what the actual scope is, because is sometimes subjective
4. Lack of semantical understanding that may help

Improving the model's ability to correctly identify the scope of these categories was our main reason to go for the DL approach with BERT. 